In [1]:
import gc
gc.collect()

217

In [2]:
# githubリポジトリをクローン
!git clone https://github.com/mk-minchul/AdaFace.git
!git clone https://github.com/afm215/python-utils.git

fatal: destination path 'AdaFace' already exists and is not an empty directory.
fatal: destination path 'python-utils' already exists and is not an empty directory.


In [3]:
# 各ライブラリのインストール
!python -m pip install --upgrade pip
!pip install gdown
!pip install -r requirements.txt
!pip install -e ./python-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.6 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [gdown]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchmetrics to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchmetrics to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 

In [4]:
!nvidia-smi

Thu Jun 19 16:20:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 2000 Ada Gene...    On  |   00000000:C2:00.0 Off |                  Off |
| 30%   27C    P8              6W /   70W |       2MiB /  16380MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!pip show torch torchvision torchaudio

Name: torch
Version: 1.13.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: nvidia-cublas-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, typing-extensions
Required-by: pytorch-lightning, torchaudio, torchmetrics, torchvision
---
Name: torchvision
Version: 0.14.1
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, requests, torch, typing-extensions
Required-by: 
---
Name: torchaudio
Version: 2.1.0+cu118
Summary: An audio package for PyTorch
Home-page: https://github.com/pytorch/audio
Author: Soumith Chintala, David Pollack, Sean Naren, Peter Goldsborough, Mo

# CONFIG

In [17]:
import os

class CONFIG:
    
    # デバックモード
    is_debug_mode = True
    max_images_num = 5

    # データセットの指定
    is_original_dataset = False
    dataset_name = 'faces_webface_112x112' if not is_original_dataset else 'original'
    dataset_dir = os.path.join("data", dataset_name)
    

In [18]:
import gdown
import zipfile

if not os.path.isdir(CONFIG.dataset_dir):
    if CONFIG.dataset_name == 'faces_umd':
        #UMD-facesをダウンロード
        file_id = '1azhEHoJjVmifuzBVKJwl-sDbLZ-Wzp4O'
        gdown.download(f"https://drive.google.com/uc?id={file_id}", f"{CONFIG.dataset_name}.zip", quiet=False)
        #zip解凍
        with zipfile.ZipFile(f"{CONFIG.dataset_name}.zip", 'r') as zip_ref:
            zip_ref.extractall("data")  # 任意の展開先フォルダ
    if CONFIG.dataset_name == 'faces_webface_112x112':
        # CASIA-webfaceをダウンロード
        file_id = '1KxNCrXzln0lal3N4JiYl9cFOIhT78y1l'
        gdown.download(f"https://drive.google.com/uc?id={file_id}", f"{CONFIG.dataset_name}.zip", quiet=False)
        #zip解凍
        with zipfile.ZipFile(f"{CONFIG.dataset_name}.zip", 'r') as zip_ref:
            zip_ref.extractall("data")  # 任意の展開先フォルダ

    if CONFIG.dataset_name == 'original':
        pass


In [19]:
!pwd

/workspace


# rec file convert to image files

In [20]:
if not os.path.exists(f"{CONFIG.dataset_dir}/imgs") and not CONFIG.is_original_dataset: # InsightFaceDataset
    !python AdaFace/convert.py --rec_path {dataset_dir} --make_image_files --make_validation_memfiles
elif not os.path.exists(f"{CONFIG.dataset_dir}") and CONFIG.is_original_dataset: # OriginalFaceDataset
    pass
else:
    pass

In [21]:
data_root = os.path.dirname(CONFIG.dataset_dir) 
dataset_dir = CONFIG.dataset_dir
dataset_img_dir = os.path.join(CONFIG.dataset_dir, "imgs")

print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}") 

 data_root: data
 dataset_dir: data/faces_webface_112x112
 dataset_img_dir:data/faces_webface_112x112/imgs


In [22]:
import shutil

debug_dir = os.path.join(data_root, "debug_dataset")
debug_img_dir = os.path.join(debug_dir, "imgs") 

if CONFIG.is_debug_mode:

    # 既存フォルダをクリア
    if os.path.isdir(debug_dir):
        shutil.rmtree(debug_dir)
    os.makedirs(debug_dir, exist_ok=True)

    # クラス名ディレクトリをソートして先頭5つ取得
    class_dirs = sorted([
        d for d in os.listdir(dataset_img_dir)
        if os.path.isdir(os.path.join(dataset_img_dir, d))
    ])[:CONFIG.max_images_num]

    # 画像データをコピー
    for cls in class_dirs:
        src = os.path.join(dataset_img_dir, cls)
        dst = os.path.join(debug_img_dir, cls)
        shutil.copytree(src, dst)
        # もしくは高速に済ませたいなら symlink:
        # os.symlink(src, dst, target_is_directory=True)

    # dataset_dirをdebug_dirに変更
    dataset_dir = debug_dir
    dataset_img_dir = debug_img_dir

else:
    pass

dataset_dir_name = os.path.basename(dataset_dir)
dataset_img_dir_name = os.path.basename(dataset_img_dir)
print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}")
print(f"▶ Using dataset_path : {dataset_dir}")

 data_root: data
 dataset_dir: data/debug_dataset
 dataset_img_dir:data/debug_dataset/imgs
▶ Using dataset_path : data/debug_dataset


In [29]:
# ImageFolder convert to recordIO
!python create_train_rec.py --img_dir {dataset_img_dir} --output_dir {dataset_dir}

# 各評価ファイルをコピー
!cp evaluations/agedb_30.bin \
   evaluations/calfw.bin \
   evaluations/cfp_ff.bin \
   evaluations/cfp_fp.bin \
   evaluations/cplfw.bin \
   evaluations/lfw.bin \
   {dataset_dir}/


100%|████████████████████████████████████████| 473/473 [00:02<00:00, 230.91it/s]
Wrote 473 images belonging to 5 classes
Record file : data/debug_dataset/train.rec
Index file  : data/debug_dataset/train.idx
List file   : data/debug_dataset/train.lst


In [38]:
!python AdaFace/convert.py --rec_path {dataset_dir} --make_validation_memfiles

loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
(12000, 3, 112, 112)
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
(12000, 3, 112, 112)
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
loading bin 13000
loading bin 14000
(14000, 3, 112, 112)
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
loading bin 13000
loading bin 14000
(14000, 3, 112, 112)
loading bin 1000

# train

In [12]:
!python AdaFace/main.py --help

usage: main.py [--data_root DATA_ROOT] [--train_data_path TRAIN_DATA_PATH]
               [--val_data_path VAL_DATA_PATH] [--use_mxrecord]
               [--train_data_subset] [--swap_color_channel] [--prefix PREFIX]
               [--gpus GPUS] [--distributed_backend {dp,ddp,ddp2}]
               [--use_16bit] [--epochs N] [--seed SEED]
               [--batch_size BATCH_SIZE] [--lr LR]
               [--lr_milestones LR_MILESTONES] [--lr_gamma LR_GAMMA]
               [--num_workers NUM_WORKERS] [--fast_dev_run] [--evaluate]
               [--resume_from_checkpoint RESUME_FROM_CHECKPOINT]
               [--start_from_model_statedict START_FROM_MODEL_STATEDICT]
               [--use_wandb] [--custom_num_class CUSTOM_NUM_CLASS]
               [--arch ARCH] [--momentum M] [--weight_decay WEIGHT_DECAY]
               [--head HEAD] [--m M] [--h H] [--s S] [--t_alpha T_ALPHA]
               [--low_res_augmentation_prob LOW_RES_AUGMENTATION_PROB]
               [--crop_augmentation_prob CRO

In [43]:
# !NUM_CLASSES=$(find "${DATA_ROOT}/${DATASET_NAME}/imgs" -mindepth 1 -maxdepth 1 -type d | wc -l)
!python AdaFace/main.py \
    --data_root {data_root} \
    --train_data_path {dataset_dir_name} \
    --val_data_path {dataset_dir_name} \
    --prefix ir101_ms1mv2_adaface \
    --gpus 1 \
    --use_16bit \
    --arch ir_101 \
    --batch_size 216 \
    --num_workers 16 \
    --epochs 2 \
    --lr_milestones 12,20,24 \
    --lr 0.1 \
    --head adaface \
    --m 0.4 \
    --h 0.333 \
    --low_res_augmentation_prob 0.2 \
    --crop_augmentation_prob 0.2 \
    --photometric_augmentation_prob 0.2

Traceback (most recent call last):
  File "/workspace/AdaFace/main.py", line 109, in <module>
    main(args)
  File "/workspace/AdaFace/main.py", line 20, in main
    trainer_mod = train_val.Trainer(**hparams)
  File "/workspace/AdaFace/train_val.py", line 18, in __init__
    self.class_num = utils.get_num_class(self.hparams)
  File "/workspace/AdaFace/utils.py", line 131, in get_num_class
    class_folders = [d for d in os.listdir(img_dir)
FileNotFoundError: [Errno 2] No such file or directory: 'data/debug_dataset'
